# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

There might be a problem with your API key? Please visit the troubleshooting notebook!


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links
# print(ed.get_contents())

['https://edwarddonner.com/',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/proficient/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/2026/02/17/ai-coder-vibe-coder-to-agentic-engineer/',
 'https://edwarddonner.com/2026/02/17/ai-coder-vibe-coder-to-agentic-engineer/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.co

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/curriculum/
https://edwarddonner.com/proficient/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://edwarddonner.com/curriculum/
https://edwarddonner.com/2026/02/17/ai-coder-vibe-coder-to-agentic-engineer/
https://edwarddonner.com/2026/02/17/ai-coder-vibe-coder-to-agentic-engineer/
https://edwarddonner.c

In [9]:
def get_links(url):
    website = Website(url)
    
    openai = OpenAI(
        api_key="sk-4e7h5BLXEa-N5FX430dmHw",
        base_url="https://llm.pubmatic.com"
    )
    
    response = openai.chat.completions.create(
        model="(paid) gpt-4o-mini", #MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/zai-org/GLM-5',
 '/MiniMaxAI/MiniMax-M2.5',
 '/Qwen/Qwen3.5-397B-A17B',
 '/Nanbeige/Nanbeige4.1-3B',
 '/openbmb/MiniCPM-SALA',
 '/models',
 '/spaces/hadadxyz/ai',
 '/spaces/itsMarco-G/reachy_phone_home',
 '/spaces/Wan-AI/Wan2.2-Animate',
 '/spaces/ACE-Step/Ace-Step-v1.5',
 '/spaces/r3gm/wan2-2-fp8da-aoti-preview',
 '/spaces',
 '/datasets/openbmb/UltraData-Math',
 '/datasets/ma-xu/fine-t2i',
 '/datasets/OpenMed/Medical-Reasoning-SFT-Mega',
 '/datasets/GD-ML/IntTravel_dataset',
 '/datasets/nvidia/SAGE-10k',
 '/datasets',
 '/join',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/inference/models',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transformers',
 '/docs/diffusers

In [11]:
get_links("https://huggingface.co")
# get_links("https://edwarddonner.com")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'community forum', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [12]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [13]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 2M+ models
Trending on
this week
Models
zai-org/GLM-5
Updated
5 days ago
•
170k
•
1.34k
MiniMaxAI/MiniMax-M2.5
Updated
2 days ago
•
40.3k
•
734
Qwen/Qwen3.5-397B-A17B
Updated
2 days ago
•
46.8k
•
647
Nanbeige/Nanbeige4.1-3B
Updated
1 day ago
•
50.9k
•
558
openbmb/MiniCPM-SALA
Updated
7 days ago
•
4.15k
•
462
Browse 2M+ models
Spaces
Running
764
Demo Playground
⚡
764
Free platform to access multiple AI mode

In [14]:
# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


In [15]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [16]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 2M+ models\nTrending on\nthis week\nModels\nzai-org/GLM-5\nUpdated\n5 days ago\n•\n170k\n•\n1.34k\nMiniMaxAI/MiniMax-M2.5\nUpdated\n2 days ago\n•\n40.3k\n•\n734\nQwen/Qwen3.5-397B-A17B\nUpdated\n2 days ago\n•\n46.8k\n•\n647\nNanbeige/Nanbeige4.1-3B\nUpdated\n1 day ago\n•\n50.9k\n•\n558\nopenbmb/MiniCPM-SALA\nUpdated\n7 days ago\n•\n4.15k\n•\n462\nBrowse 2M+ models\nSpaces\nRunning\n764\nDemo Playground\n⚡\n764\nFree platform to acce

In [21]:
def create_brochure(company_name, url):
    openai = OpenAI(
        api_key="sk-4e7h5BLXEa-N5FX430dmHw",
        base_url="https://llm.pubmatic.com"
    )
    
    response = openai.chat.completions.create(
        model= "(paid) gpt-4o-mini", #MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [22]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'community discussions', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}


# Welcome to Hugging Face!

**The AI Community Building the Future**  
Where every hug is a byte closer to brilliance!

---

## Who Are We?
At Hugging Face, we're not just about cuddly names and adorable mascots; we're the fearless squad of AI enthusiasts paving the way for cutting-edge machine learning. Imagine if a tech brainstorming session collided with a pajama party—this is us at Hugging Face! We provide a collaborative platform where both seasoned pros and curious newbies can mix models, datasets, and applications like the ultimate tech smoothie. 

### Our Offerings
- **Models Galore**! (2M+ to be exact): Check out our trending models like "zai-org/GLM-5," which will make your AI dreams take flight.
- **Datasets**: 500k options! More choices than a buffet at Las Vegas (and much less messy).
- **Spaces**: A free playground to showcase your AI creations! (Warning: May cause spontaneous marvel from onlookers).

## Our Culture
### Come for the AI, Stay for the Feelings!
Our community members give virtual hugs, share knowledge, and build amazing tech. We believe that collaboration makes the best hug—err, we mean, solutions! Plus, we’re an open-source loving family, where everyone gets a seat at the model table... as long as you bring snacks. 🥳

---

## Who Are Our Customers?
Over **50,000 organizations** (including tech giants like Google, Microsoft, and Amazon) trust Hugging Face like kids trust their stuffed animals. So whether you're starting up or a unicorn in the making, we've got your back.

## Career Opportunities
### Hugging Face: Where Careers Flourish like a Sunflower in AI
Looking for a way to bring your tech wizardry to life? Join us! We’re on the lookout for fresh talent who can navigate the wild waters of AI development and want to do it in a cozy, community-driven environment. Job openings are like our models—constantly updated, with new chances blooming every week!

---

## Why Hugging Face?
- **Innovation**: We don’t just ride the wave of AI; we create the waves! 🌊
- **Support**: Need help? Our dedicated team is more responsive than a cat to a laser pointer.
- **Community**: Collaboration is in our DNA. Join your fellow brainiacs in shaping the future of AI.

---

### Get Started Today!
**Hugging Face—where tech dreams come true!** Whether you're here to explore, collaborate, or revolutionize the industry, we promise you’ll never feel alone. Sign up today, and let’s hug it out with some AI!

![Hugging Face Logo](https://huggingface.co/static/images/logo.svg)

---

**Contact Us Today!** We'd love to hear from you or just give you a virtual hug.  
[Hugging Face](https://huggingface.co) | Follow us on social media!  
📧 info@huggingface.co | 🐦 @HuggingFace  

### *Together, let's build a hugging great future!*

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [23]:
def stream_brochure(company_name, url):
    openai = OpenAI(
        api_key="sk-4e7h5BLXEa-N5FX430dmHw",
        base_url="https://llm.pubmatic.com"
    )
    
    stream = openai.chat.completions.create(
        model="(paid) gpt-4o-mini", #MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    #-------------------
    # prints without markdown
    # for chunk in stream:
    #     print(chunk.choices[0].delta.content or '', end='')
    #-------------------

    # print with markdown
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [24]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'github page', 'url': 'https://github.com/huggingface'}, {'type': 'linkedin page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}



# 🎉 Welcome to Hugging Face! 🎉

## The AI Community That Hugs Back!

At Hugging Face, we're like that friend who always supports you when you’re feeling low (as long as you share your models and datasets). We are building the future of AI, not by ourselves, but together—with YOU!

### 🤖 What We Do

- **Explore Models**: Browse our **2M+** top-notch AI models. Seriously, that’s more than your average online sock store!
- **Datasets Galore**: Tired of data that doesn’t make sense? Dive into our **500k+ datasets**. No need to go on a wild goose chase for the right info!
- **Spaces**: We provide **764** demo playgrounds for your now not-so-imaginary AI dreams. Yes, it’s *that* easy to kickstart your imagination (with slightly fewer messy Scribbles...).

### 🌐 Join the Hugging Family!

With over **50,000 organizations** snuggling up, including AI giants like Google and Microsoft, you’ll be in *some* pretty heavy company! We also boast a community that keeps things lively with regular updates on models, collaborative projects, and a lot of **‘Hug it Out’** sessions.

### 🏢 Careers at Hugging Face

We’re hunting for passionate AI enthusiasts to join our diverse team! Want to help shape the future? Get ready for:

- **Snackalicious Office**: Remember to bring your best lunchbox because we believe in intellectual nourishment!
- **Chill Culture**: We promote collaboration over competition. You can share memes as freely as you share your datasets (just keep the cat videos to a minimum).
- **Flexible Hours**: Wake up at noon? More power to you! Just be sure to hit that ‘Online’ switch when you finally roll out of bed.

### 📈 Investing in Hugging Face

Looking to invest? You’d be putting your money in the huggiest AI mix around! Join us as we open doors to new heights in machine learning. Your golden ticket to the future of AI is just a few clicks away! 🎟️

### 🤝 So, What Are You Waiting For? 

Join the Hugging Community today—where the AI future isn't just built; it’s lovingly nurtured! Because here, everyone’s a big deal... including your models! 

-  **Explore our website 📅**: [Hugging Face](https://huggingface.co) 
-  **Follow us on social media 📲**: Join the conversation!
-  **Sign up for a free account now! 🚀**

Time to wrap it up and get hugging! 💖
 

Feel free to let me know if you need any changes or additional details!

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>